In [ ]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
from scipy import stats
import scipy.signal
import IPython
from IPython.display import Audio, IFrame, display
import numpy as np
import io
import librosa
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data1, sr = librosa.load('../input/birdsong-recognition/train_audio/aldfly/XC2628.mp3', mono=True, offset=0)
data2, sr = librosa.load('../input/birdsong-recognition/train_audio/bkcchi/XC278350.mp3', mono=True, offset=0)
data3, sr = librosa.load('../input/birdsong-recognition/train_audio/rthhum/XC319183.mp3', mono=True, offset=0)
data4, sr = librosa.load('../input/birdsong-recognition/train_audio/tunswa/XC389638.mp3', mono=True, offset=0)

In [ ]:
display(Audio(data1, rate=sr))
display(Audio(data2, rate=sr))
display(Audio(data3, rate=sr))
display(Audio(data4, rate=sr))

In [ ]:
def calculateThreshold(data,sr = 22050):
    window = int(len(data)/sr)
    dataSeries=pd.Series(data.tolist())
    runningMin=dataSeries.rolling(window).min()
    runningMax=dataSeries.rolling(window).max()
    minMax=runningMax-runningMin
    minMax=minMax.tolist()
    minMax=[minMax for minMax in minMax if str(minMax) != 'nan']
    threshold=np.quantile(minMax,0.25)
    return(threshold)

In [ ]:
def removeDataTwoBands(arr, threshold):
    index = 0;
    for i in range(len(arr)):
        if arr[i]>threshold or arr[i]<-threshold:
            continue
        else:
            arr[index] = arr[i]
            index += 1
    return arr[0:index]

In [ ]:
def isolateNoise(data, threshold):
    toList = data.tolist()
    removeHighIntensity = removeDataTwoBands(toList, threshold)
    return np.array(removeHighIntensity, dtype=float)

In [ ]:
def getHighIntensityFrequencies(spectData):
    frequencies = []
    for i in range(186,spectData.shape[0]):
        for j in range(spectData.shape[1]):
            if spectData[i,j] > 0:
                frequencies.append(i)
    return(frequencies)

In [ ]:
def getBandLimits(spectData):
    highIntensityFreq = getHighIntensityFrequencies(spectData)
    highIntensityHz=list(map(lambda x: x*11000/spectData.shape[0],highIntensityFreq))
    center = float(stats.mode(highIntensityHz)[0])
    sd = np.std(highIntensityHz)
    lowcut = center - sd
    highcut = center + sd
    return(lowcut,highcut)

In [ ]:
from scipy.signal import butter, lfilter

def bandpassFilter(data, lowcut, highcut, sr, order=5):
    nyq = 0.5 * sr
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    result = lfilter(b, a, data)
    return result

In [ ]:
def removeDataOneBand(arr, threshold):
    index = 0;
    for i in range(len(arr)):
        if arr[i]<threshold and arr[i]>-threshold:
            continue
        else:
            arr[index] = arr[i]
            index += 1
    return arr[0:index]

In [ ]:
def isolateBirdCall(data, threshold):
    toList = data.tolist()
    result = removeDataOneBand(toList, threshold)
    return np.array(result, dtype=float)

In [ ]:
def fragmentData(data):
    if len(data)>sr:
        threshold=calculateThreshold(data)
        birdCall = isolateBirdCall(data, threshold)
    else:
        birdCall = data
    return display(Audio(birdCall, rate=sr))

In [ ]:
fragmentData(data1)
fragmentData(data2)
fragmentData(data3)
fragmentData(data4)